# FABRIC API Example: Create a Simple Slice


## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

In [ ]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering project_name and scope parameters in the refresh call below.


In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Configure Slice Parameters



In [ ]:
slice_name='my_simple_slice'
node_name='node1'
site='LBNL'
image_name='default_centos_8'

## Build the Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
# Create topology
myExperiment = ExperimentTopology()

# Add node
myNode = myExperiment.add_node(name=node_name, site=site)

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=16, disk=100)

# Set Properties
myNode.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)

## Submit the Slice

In [ ]:
# Generate Slice Graph
slice_graph = myExperiment.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

In [ ]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

print(f"Slice ID: {slice_id}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))